In [39]:
# Result structure:
# |--result
#    |--[*DataSet*](Irish_2010&London_2013)
#    |  |--clustering
#    |     |--point
#    |        |--hierarchical
#    |        |  |--euclidean
#    |        |  |--cityblock
#    |        |  |--DTW
#    |        |--kmeans


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering

In [40]:
data_set = 'Irish_2010'
attr = pd.read_csv('../data/' + data_set + '_attr_final.csv')

In [3]:
# K-means clustering

for num_clusters in range(1, 11):
    labels = []
    for month in tqdm(range(12)):

        X = []
        for i in range(len(attr)):
            id = attr['ID'][i]
            df = pd.read_csv('../data/' + data_set + '_profiles/' + str(id) + '.csv', header = None).values
            X.append(df[month])
        X = np.array(X)
        X = (X - np.min(X))/(np.max(X) - np.min(X))

        kmeans = KMeans(n_clusters=num_clusters, init='k-means++', n_init=10, max_iter=300, tol=0.0001, precompute_distances=True, 
                        verbose=0, random_state=None, copy_x=True, n_jobs=None, algorithm='full').fit(X)
        labels.append(kmeans.labels_)

    pd.DataFrame(np.array(labels).T).to_csv('../result/' + data_set + '/clustering/point/kmeans/n_clusters_' + str(num_clusters) + '.csv', header=None, index=False)

100%|██████████| 12/12 [00:25<00:00,  2.09s/it]


In [36]:
# Hierarchical clustering (euclidean & cityblock)

dist = 'cityblock'
for num_clusters in range(1, 11):
    labels = []
    for month in tqdm(range(12)):

        X = []
        for i in range(len(attr)):
            id = attr['ID'][i]
            df = pd.read_csv('../data/' + data_set + '_profiles/' + str(id) + '.csv', header = None).values
            X.append(df[month])
        X = np.array(X)
        X = (X - np.min(X))/(np.max(X) - np.min(X))

        hierarchical = AgglomerativeClustering(n_clusters=num_clusters, affinity=dist, memory=None, connectivity=None, 
                                               compute_full_tree='auto', linkage='complete', pooling_func='deprecated', distance_threshold=None).fit(X)
        labels.append(hierarchical.labels_)

    pd.DataFrame(np.array(labels).T).to_csv('../result/' + data_set + '/clustering/point/hierarchical/' + dist + '/n_clusters_' + str(num_clusters) + '.csv', header=None, index=False)

100%|██████████| 12/12 [00:47<00:00,  4.01s/it]


In [41]:
# Hierarchical clustering (DTW)

dist = 'DTW'
for num_clusters in range(1, 11):
    labels = []
    for month in tqdm(range(12)):

        mat = pd.read_csv('../result/' + data_set + '/clustering/point/hierarchical/' + dist + '/mat_month_' + str(month+1) + '.csv', header=None).values
        hierarchical = AgglomerativeClustering(n_clusters=num_clusters, affinity='precomputed', memory=None, connectivity=None, 
                                               compute_full_tree='auto', linkage='complete', pooling_func='deprecated', distance_threshold=None).fit(mat)
        labels.append(hierarchical.labels_)

    pd.DataFrame(np.array(labels).T).to_csv('../result/' + data_set + '/clustering/point/hierarchical/' + dist + '/n_clusters_' + str(num_clusters) + '.csv', header=None, index=False)

100%|██████████| 12/12 [00:02<00:00,  4.10it/s]


## Clustering with Acorn groups

In [52]:
# Load attributes
data_set = 'London_2013'
attr = pd.read_csv('../data/' + data_set + '_attr_final.csv')

In [53]:
# Transform to 0-1
X_extra = pd.get_dummies(attr['Cate'], prefix='Cate').values

In [9]:
# K-means clustering

for num_clusters in range(1, 11):
    labels = []
    for month in tqdm(range(12)):

        X = []
        for i in range(len(attr)):
            id = attr['ID'][i]
            df = pd.read_csv('../data/' + data_set + '_profiles/' + str(id) + '.csv', header = None).values
            X.append(df[month])
        X = np.array(X)
        X = (X - np.min(X))/(np.max(X) - np.min(X))
        X = np.hstack((X, X_extra))

        kmeans = KMeans(n_clusters=num_clusters, init='k-means++', n_init=10, max_iter=300, tol=0.0001, precompute_distances=True, 
                        verbose=0, random_state=None, copy_x=True, n_jobs=None, algorithm='full').fit(X)
        labels.append(kmeans.labels_)

    pd.DataFrame(np.array(labels).T).to_csv('../result/' + data_set + '/clustering/point/kmeans/n_clusters_' + str(num_clusters) + '_acorn.csv', header=None, index=False)

100%|██████████| 12/12 [00:47<00:00,  3.98s/it]


In [54]:
# Hierarchical clustering (euclidean & cityblock)

dist = 'cityblock'
for num_clusters in range(1, 11):
    labels = []
    for month in tqdm(range(12)):

        X = []
        for i in range(len(attr)):
            id = attr['ID'][i]
            df = pd.read_csv('../data/' + data_set + '_profiles/' + str(id) + '.csv', header = None).values
            X.append(df[month])
        X = np.array(X)
        X = (X - np.min(X))/(np.max(X) - np.min(X))
        X = np.hstack((X, X_extra))

        hierarchical = AgglomerativeClustering(n_clusters=num_clusters, affinity=dist, memory=None, connectivity=None, 
                                               compute_full_tree='auto', linkage='complete', pooling_func='deprecated', distance_threshold=None).fit(X)
        labels.append(hierarchical.labels_)

    pd.DataFrame(np.array(labels).T).to_csv('../result/' + data_set + '/clustering/point/hierarchical/' + dist + '/n_clusters_' + str(num_clusters) + '_acorn.csv', header=None, index=False)

100%|██████████| 12/12 [00:56<00:00,  4.68s/it]
